### **Prediction with Virsorter2** 

In [1]:
vs2_command = 'virsorter run -w genome_Xf1_out_dir/ -i genome_Xf1.fna -j 10 all'
print(vs2_command)

virsorter run -w genome_Xf1_out_dir/ -i genome_Xf1.fna -j 10 all


### **Prediction with Inovirus_detector** 

In [4]:
inovirus_comand1 = str('perl Identify_candidate_fragments_from_gbk.pl -g genome_Xf1.gbk '+\
                       '-p /Inovirus_detector/Pfam-A.hmm '+\
                       '-sp /inovirus_detector/bin/signalp '+\
                       '-th /inovirus_detector/bin/tmhmm '+\
                       '-d Inovirus_detector/Inovirus_db')
inovirus_comand2 = 'perl Get_inovirus_prediction_score_from_gff_fragments.pl'+\
                   '-i candidate_gff_from_inovirus_comand1 -f candidate_fna_from_inovirus_comand1'
print(inovirus_comand1)
print()
print(inovirus_comand2)

perl Identify_candidate_fragments_from_gbk.pl -g genome_Xf1.gbk -p /Inovirus_detector/Pfam-A.hmm -sp /inovirus_detector/bin/signalp -th /inovirus_detector/bin/tmhmm -d Inovirus_detector/Inovirus_db

perl Get_inovirus_prediction_score_from_gff_fragments.pl-i candidate_gff_from_inovirus_comand1 -f candidate_fna_from_inovirus_comand1


### **Prediction with PHASTER**  

In [7]:
# Phaster server was used as follows:
phaster_web = 'https://phaster.ca/'
print(phaster_web)

https://phaster.ca/


### **Prediction with SeqWord Sniffer**

In [8]:
seqword_sniffer_comand = 'SeqWordSniffer.py -f genome_Xf1.fna'
print(seqword_sniffer_comand)

SeqWordSniffer.py -f genome_Xf1.fna


### **Prediction with GIPSy**

In [9]:
# It was used the User Interfase of GIPSy
gipsy_manual = 'https://www.bioinformatics.org/download/gipsy/Manual.pdf'
print(gipsy_manual)

https://www.bioinformatics.org/download/gipsy/Manual.pdf


### **Prediction with ISEScan** 

In [10]:
isescan_comand = 'isescan.py --nthread 4 --seqfile genome_Xf1.fna --output genome_Xf1_result'
print(isescan_comand)

isescan.py --nthread 4 --seqfile genome_Xf1.fna --output genome_Xf1_result


### Mobilome Integration 

In [13]:
# Models of the matrix organization to the next function
# mge1_matrix = [['Temecula1_phCons-1', 'NC_004556.1', 446956, 480076, 'Temecula1_Phaster-1$intact'],
#               ['Temecula1_phCons-2', 'NC_004556.1', 942607, 968337, 'Temecula1_Phaster-2$incomplete'],
#               ['Temecula1_phCons-3', 'NC_004556.1', 1120090, 1125199, 'Temecula1_Phaster-3$questionable'],
#               ['Temecula1_phCons-4', 'NC_004556.1', 1127842, 1141158, 'Temecula1_Inov-1$att:True'],
#               ['Temecula1_phCons-5', 'NC_004556.1', 1161143, 1180387, 'Temecula1_Phaster-5$questionable'],
#               ['Temecula1_phCons-6', 'NC_004556.1', 1194458, 1220728, 'Temecula1_Phaster-6$intact'],...]

# mge2_matrix = [['Temecula1_giCons-1', 'NC_004556.1', 1, 18099, 'Temecula1_sws-1$NA'],
#                ['Temecula1_giCons-2', 'NC_004556.1', 57500, 79099, 'Temecula1_sws-2$NA'],
#                ['Temecula1_giCons-3', 'NC_004556.1', 154000, 172099, 'Temecula1_sws-3$NA'],
#                ['Temecula1_giCons-4', 'NC_004556.1', 453610, 468384, 'Temecula1_gipsy-1$patho'],
#                ['Temecula1_giCons-5', 'NC_004556.1', 1079500, 1101099, 'Temecula1_sws-5$NA'],
#                ['Temecula1_giCons-6', 'NC_004556.1', 1128000, 1150099, 'Temecula1_sws-6$NA'], ... ]

In [14]:
def integration_prophages_genomicisland(mge1_matrix , mge2_matrix):
    ''' This function takes one matrix of prophages results and one matrix of genomic island results,
        and return a consensus prioritizing the prophage regions '''
    # Retriving locations
    dic_ph_considered = {}
    dic_gi_considered = {}
    for gi_list in mge2_matrix:         
        for ph_list in mge1_matrix:      
            if gi_list[1] == ph_list[1]:
                id1,loc1 = ph_list[:2]
                id2,loc2 = gi_list[:2]
                
                s1,e1,f1 = ph_list[2],ph_list[3],ph_list[4]
                s2,e2,f2 = gi_list[2],gi_list[3],gi_list[4]
                l1 , l2 = e1-s1+1 , e2-s2+1
                
                # CASE 1: gi is into ph
                if (s2 >= s1 and e2 <= e1):
                    dic_gi_considered[id2+'-case1'] = 'not'
                    if (not id1 in list(i.split('-case')[0] for i in list(dic_ph_considered))):
                        make = [10,(id1),(loc1,loc2),(s1,e1),(f1)]
                        dic_ph_considered[id1+'-case1'] = make
                    
                # CASE 2: ph is into gi
                elif (s2 <= s1 and s2 <= e1) and (e2 > s1 and e2 >= e1):
                    dic_gi_considered[id2+'-case2'] = 'not'
                    if (not id1 in list(i.split('-case')[0] for i in list(dic_ph_considered))):
                        make = [20,(id1),(loc1,loc2),(s1,e1),(f1)]
                        dic_ph_considered[id1+'-case2'] = make
                    
                # CASE 3: gi at the left
                elif (s2 <= s1) and (e2 > s1) and (e2 < e1):
                    dic_gi_considered[id2+'-case3'] = 'not'
                    if (not id1 in list(i.split('-case')[0] for i in list(dic_ph_considered))):
                        perc_overlap = round((e2 - s1)/ l1 * 100 , 2)
                        if perc_overlap <= 15:
                            make = [30,(id2),(loc1,loc2),(s2,s1-1000),(f2)]
                            dic_gi_considered[id2+'-case3'] = make
                            make = [30,(id1),(loc1,loc2),(s1,e1),(f1)]
                            dic_ph_considered[id1+'-case3'] = make
                        else:
                            make = [30,(id1),(loc1,loc2),(s1,e1),(f1)]
                            dic_ph_considered[id1+'-case3'] = make
                        
                    else:
                        pass

                # CASE 4: mge2 at the right 
                elif (s2 > s1) and (s2 < e1) and (e2 > e1):
                    #print('right',id1,s1,e1,id2,s2,e2)
                    dic_gi_considered[id2+'-case4'] = 'not'
                    if (not id1 in list(i.split('-case')[0] for i in list(dic_ph_considered))):
                        perc_overlap = round((e1 - s2)/ l1 * 100 , 2)
                        if (perc_overlap <= 15):
                            make = [40,(id2),(loc1,loc2),(e1+1000,e2),(f2)]
                            dic_gi_considered[id2+'-case4'] = make
                            make = [40,(id1),(loc1,loc2),(s1,e1),(f1)]
                            dic_ph_considered[id1+'-case4'] = make
                        else:
                            make = [40,(id1),(loc1,loc2),(s1,e1),(f1)]
                            dic_ph_considered[id1+'-case4'] = make
                    else:
                        perc_overlap = round((e1 - s2)/ l1 * 100 , 2)
                        if perc_overlap <= 15:
                            print(id2,perc_overlap)
                            make = [40,(id2),(loc1,loc2),(e1+1000,e2),(f2)]
                            dic_gi_considered[id1+'-case4'] = make
                        else:
                            pass
                else:
                    pass
            else:
                pass
    
    for gi_list in mge2_matrix: 
        for ph_list in mge1_matrix: 
            # CASE 5: ph is not in overlaping        
            if (not ph_list[0] in list(i.split('-case')[0] for i in list(dic_ph_considered))):
                make = [50,(ph_list[0]),('Non-loc1',ph_list[1]),
                        (ph_list[2],ph_list[3]),(ph_list[4])]
                dic_ph_considered[ph_list[0]+'-case5'] = make
            # CASE 6: gi is not in overlaping
            if (not gi_list[0] in list(i.split('-case')[0] for i in list(dic_gi_considered))):
                make = [60,(gi_list[0]) , (gi_list[1],'Non-loc2'),
                         (gi_list[2],gi_list[3]) , (gi_list[4])]
                dic_gi_considered[gi_list[0]+'-case6'] = make
    
    dic_ph_considered.update(dic_gi_considered)
    # making general list of the consensus
    consensus = []
    for k,v in dic_ph_considered.items():
        if v != 'not':
            cons_colum_1 = v[1]
            cons_colum_2 = ''
            for loc in v[2]:
                if not loc.startswith('Non-loc'):
                    cons_colum_2 = loc
            cons_colum_3 = v[3][0]
            cons_colum_4 = v[3][1]
            cons_colum_5 = v[4]
            consensus.append([cons_colum_1,cons_colum_2,cons_colum_3,cons_colum_4,cons_colum_5])
    return consensus

In [15]:
# Models of the matrix organization to the next function
# ph_gi_matrix = [['Temecula1_phCons-1', 'NC_004556.1', 446956, 480076, 'Temecula1_Phaster-1$intact'],
#                 ['Temecula1_phCons-4', 'NC_004556.1', 1127842, 1141158, 'Temecula1_Inov-1$att:True'],
#                 ['Temecula1_phCons-5', 'NC_004556.1', 1161143, 1180387, 'Temecula1_Phaster-5$questionable'],
#                 ...
#                 ['Temecula1_giCons-5', 'NC_004556.1', 1079500, 1101099, 'Temecula1_sws-5$NA'],
#                 ['Temecula1_giCons-10', 'NC_004556.1', 1441831, 1458553, 'Temecula1_sws-8$NA'],
#                 ['Temecula1_giCons-11', 'NC_004556.1', 1645000, 1662599, 'Temecula1_sws-9$NA'], ...]

# is_matrix = [['Temecula1_isCons-1', 'NC_004556.1', 180, 1563, 'Temecula1_isescan-1$IS21'],
#              ['Temecula1_isCons-2', 'NC_004556.1', 1155657, 1156053, 'Temecula1_isescan-2$IS200/IS605'],
#              ['Temecula1_isCons-3', 'NC_004556.1', 1386787, 1388482, 'Temecula1_isescan-3$IS200/IS605'],
#              ['Temecula1_isCons-4', 'NC_004556.1', 1560835, 1561745, 'Temecula1_isescan-4$IS6'],
#              ['Temecula1_isCons-5', 'NC_004556.1', 1907576, 1909682, 'Temecula1_isescan-5$IS1595']]

In [16]:
def integration_prophages_genomicisland_insertionsequences(ph_gi_matrix , is_matrix):
    ''' This function takes the results of integration_prophages_genomicisland() function,
        and integrate the insertion sequences regions '''
    dic_is_considered = {}
    for is_list in is_matrix:
        for ph_gi_list in ph_gi_matrix:
            if is_list[1] == ph_gi_list[1]:
                id1,loc1 = ph_gi_list[:2]
                id2,loc2 = is_list[:2]
                s1,e1,f1 = ph_gi_list[2],ph_gi_list[3],ph_gi_list[4]
                s2,e2,f2 = is_list[2],is_list[3],is_list[4]

                if (not id2 in list(i.split('-into-')[0] for i in list(dic_is_considered))):
                    if  (s2 >= s1 and e2 <= e1) or \
                        ((s2 <= s1) and (e2 > s1) and (e2 < e1)) or \
                        ((s2 >  s1) and (s2 < e1) and (e2 > e1)) :
                        if '_ph' in id1:
                            dic_is_considered[id2+'-into-ph'] = is_list[:-1] + [is_list[-1]+'$into-ph']
                        else:
                            dic_is_considered[id2+'-into-gi'] = is_list[:-1] + [is_list[-1]+'$into-gi']
            
    for is_list in is_matrix:
        if (not is_list[0] in list(i.split('-into-')[0] for i in list(dic_is_considered))):
            if dic_contig_type[is_list[1]] == 'pls':
                dic_is_considered[is_list[0]+'-into-pls'] = is_list[:-1] + [is_list[-1]+'$into-pls']
            elif dic_contig_type[is_list[1]] == 'chr':
                dic_is_considered[is_list[0]+'-into-chr'] = is_list[:-1] + [is_list[-1]+'$into-chr']
            
    consensus = []        
    for ph_gi in ph_gi_matrix:
        if dic_contig_type[ph_gi[1]] == 'pls':
            ph_gi[4] = ph_gi[4] + '$into-pls'
        elif dic_contig_type[ph_gi[1]] == 'chr':
            ph_gi[4] = ph_gi[4] + '$into-chr'
        consensus.append(ph_gi)
    for k,v in dic_is_considered.items():
        consensus.append(v)
    return consensus